In [2]:
import gzip
from collections import defaultdict
import math
import scipy
import scipy.optimize
from sklearn import svm
from sklearn import linear_model
import numpy as np
import string
import random
import matplotlib.pyplot as plt
import tensorflow as tf
import json
import csv
import tensorflow as tf

### Item Recommendation with Metadata Dataset

In [3]:
def parseData(path):
    # count = 0
    for l in gzip.open(path):
        l = json.loads(l)
        u = l['gmap_id']
        # count += 1
        yield u,l

In [4]:
allBusiness = defaultdict(dict)
for u,l in parseData("input/meta-California.json.gz"):
    allBusiness[u] = l

In [5]:
allBusiness["0x80d954f844e56335:0x1e73613cd10bf321"]

{'name': 'Golden Days Tattoo',
 'address': 'Golden Days Tattoo, 2848 El Cajon Blvd, San Diego, CA 92104',
 'gmap_id': '0x80d954f844e56335:0x1e73613cd10bf321',
 'description': None,
 'latitude': 32.7555647,
 'longitude': -117.1328405,
 'category': ['Tattoo shop'],
 'avg_rating': 4.9,
 'num_of_reviews': 38,
 'price': None,
 'hours': [['Monday', '12–8PM'],
  ['Tuesday', '12–8PM'],
  ['Wednesday', '12–8PM'],
  ['Thursday', '12–8PM'],
  ['Friday', '12–8PM'],
  ['Saturday', '12–8PM'],
  ['Sunday', '12–8PM']],
 'MISC': {'Service options': ['Delivery'],
  'Accessibility': ['Wheelchair accessible entrance']},
 'state': 'Closed ⋅ Opens 12PM Tue',
 'relative_results': ['0x80d954fefa291213:0x8ac9a253251e67b0',
  '0x80d954f69eb55427:0x2bdcd81dcbc83d97',
  '0x80d95481d33d0021:0x8da057c5201dfd5e',
  '0x80d954ffca090fdf:0xa1fdd35af1348519',
  '0x80d954539b797a81:0xa448dae9b06074d2'],
 'url': 'https://www.google.com/maps/place//data=!4m2!3m1!1s0x80d954f844e56335:0x1e73613cd10bf321?authuser=-1&hl=en&gl=

In [6]:
len(allBusiness)

513134

In [7]:
allBusiness_processed = defaultdict(dict)
for b in allBusiness:
    if 'relative_results' in allBusiness[b] and allBusiness[b]['relative_results'] != None:
        recs = []
        for rec in allBusiness[b]['relative_results']:
            if rec in allBusiness:
                recs.append(rec)
        rec_length = len(recs)
        if rec_length >= 2:
            allBusiness_processed[b] = allBusiness[b]
            allBusiness_processed[b]['processed_relative_result'] = recs
            allBusiness_processed[b]['num_recommendation'] = rec_length

In [8]:
len(allBusiness_processed)

362575

In [9]:
allBusiness_processed["0x80c2c778e3b73d33:0xbdc58662a4a97d49"]

{'name': 'San Soo Dang',
 'address': 'San Soo Dang, 761 S Vermont Ave, Los Angeles, CA 90005',
 'gmap_id': '0x80c2c778e3b73d33:0xbdc58662a4a97d49',
 'description': None,
 'latitude': 34.0580917,
 'longitude': -118.2921295,
 'category': ['Korean restaurant'],
 'avg_rating': 4.4,
 'num_of_reviews': 18,
 'price': None,
 'hours': [['Thursday', '6:30AM–6PM'],
  ['Friday', '6:30AM–6PM'],
  ['Saturday', '6:30AM–6PM'],
  ['Sunday', '7AM–12PM'],
  ['Monday', 'Closed'],
  ['Tuesday', '6:30AM–6PM'],
  ['Wednesday', '6:30AM–6PM']],
 'MISC': {'Service options': ['Takeout', 'Dine-in', 'Delivery'],
  'Accessibility': ['Wheelchair accessible entrance'],
  'Offerings': ['Comfort food'],
  'Amenities': ['Good for kids'],
  'Atmosphere': ['Casual']},
 'state': 'Open ⋅ Closes 6PM',
 'relative_results': ['0x80c2c78249aba68f:0x35bf16ce61be751d',
  '0x80c2c79998f75fff:0xd7ca5c67e96fb778',
  '0x80c2b899146d7507:0xf4162c12c9cf65f8',
  '0x80c2c77f2d419951:0x26285631b21e324c',
  '0x80c2b8add9016015:0x15836f81a96

## Rating Prediction

In [10]:
def parseData_review(path):
    count = 0
    for l in gzip.open(path):
        if count % 70 == 69: # only get a subset of the data
        # if count < 10:
            d = json.loads(l)
            # u = d['user_id']
            # t = d['time']
            # r = d['rating']
            # w = d['text']
            # g = d['gmap_id']
            # b = allBusiness[g]
            # print(count)
            yield d
            # yield d,u,t,g,r,w,b
        count += 1

In [11]:
review_data = list(parseData_review("input/review-California.json.gz"))

In [12]:
len(review_data)

1007571

In [13]:
review_processed = []
for review in review_data:
    if review['rating'] != None and review['text'] != None:
        review_processed.append(review)

In [14]:
len(review_processed)

555133

In [13]:
review_data[4300]

{'user_id': '113621160970336192329',
 'name': 'Holly Upshaw',
 'time': 1514599896095,
 'rating': 4,
 'text': "I think what they're doing for the homeless is awesome",
 'pics': None,
 'resp': None,
 'gmap_id': '0x54d15565d42b6877:0xa3f308039d1f2e12'}

In [15]:
for review in review_processed:
    if review['rating'] >= 3:
        review['sentiment'] = 1 # positive review
    else:
        review['sentiment'] = -1 # negative review

In [16]:
review_processed[0]

{'user_id': '103484871430798519961',
 'name': 'Mandy Hosford',
 'time': 1550248197033,
 'rating': 5,
 'text': 'Black Tie was so professional and helpful. It was my first time at Mammoth and I needed EVERYTHING except socks. Freals. Max was amazing and incredibly helpful and was totally invested in ensuring I had the best kit for me and the mountain. On a powder day I maybe would’ve understood if the staff wanted to be anywhere but work but it didn’t show at all. Seemed like these guys are psyched for others to get after it and I was way impressed.',
 'pics': None,
 'resp': {'time': 1573506420134,
  'text': 'Yes, Max is amazing and we love to hear positive feedback about our awesome staff. Our team is all very passionate about the mountain lifestyle and we love the opportunity to share that passion with our guests. Thanks for giving us the chance to earn your business.'},
 'gmap_id': '0x80960c29f2e3bf29:0x4b291f0d275a5699',
 'sentiment': 1}

In [17]:
ratings = [d['rating'] for d in review_processed if d['rating'] != None]

In [18]:
sum(ratings) / len(ratings)

4.236316342209885

In [ ]:
# random.shuffle(review_data)

In [49]:
len([x for x in review_processed if x['rating'] == None])

0

## Trigrams

In [17]:
wordCount = defaultdict(int)
punctuation = set(string.punctuation)
for d in review_processed[:50000]: # Strictly, should just use the *training* data to extract word counts
    r = d['text']
    # if r != None:
    r = r.lower()
    # if r != None:
    r = ''.join([c for c in r if not c in punctuation])
    # if r != None:
    ws = r.split()
    ws3 = [' '.join(x) for x in list(zip(ws[:-2], ws[1:-1], ws[2:]))]
    for w in ws3: # just trigrams
        wordCount[w] += 1

counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()

words = [x[1] for x in counts[:1000]] # top 2K n-grams
len(words) # 19K unique words

1000

In [18]:
wordID = dict(zip(words, range(len(words))))
wordSet = set(words)

In [19]:
def feature(datum):
    feat = [0]*len(words)
    r = datum['text']
    # if r != None:
    r = r.lower()
    # if r != None:
    r = ''.join([c for c in r if not c in punctuation])
    # if r != None:
    ws = r.split()
    ws3 = [' '.join(x) for x in list(zip(ws[:-2], ws[1:-1], ws[2:]))]
    for w in ws3:
        if w in words:
            feat[wordID[w]] += 1
    feat.append(1) # offset
    return feat

In [125]:
positive_reviews = []
negative_reviews = []

for d in review_processed:
    if d['rating'] > 3:
        d['sentiment'] = 1
        positive_reviews.append(d)
    else:
        d['sentiment'] = -1
        negative_reviews.append(d)

In [126]:
len(positive_reviews), len(negative_reviews)

(447820, 107313)

In [127]:
random.shuffle(positive_reviews)
random.shuffle(negative_reviews)

In [190]:
positive_reviews_million = positive_reviews[:100000]
negative_reviews_million = negative_reviews[:100000]

In [191]:
len(positive_reviews_million), len(negative_reviews_million)

(100000, 100000)

In [192]:
train_labels = positive_reviews_million[:80000] + negative_reviews_million[:80000]
valid_labels = positive_reviews_million[80000:] + negative_reviews_million[80000:]

In [195]:
random.shuffle(train_labels)
random.shuffle(valid_labels)

In [196]:
Xtrain = [feature(d) for d in train_labels]
Xvalid = [feature(d) for d in valid_labels]

In [197]:
ytrain = [d['rating'] for d in train_labels]
yvalid = [d['rating'] for d in valid_labels]

In [193]:
# mixed_reviews = positive_reviews_million + negative_reviews_million
# random.shuffle(mixed_reviews)

In [194]:
# mixed_reviews[5]

In [107]:
# X = [feature(d) for d in mixed_reviews] # review_processed does not have None

In [144]:
# y = [d['rating'] for d in mixed_reviews]

In [159]:
# Ntrain = 160000
# Nvalid = 40000

In [160]:
# Xtrain = X[:Ntrain]
# Xvalid = X[Ntrain:Ntrain+Nvalid]
# ytrain = y[:Ntrain]
# yvalid = y[Ntrain:Ntrain+Nvalid]
# valid_reviews = mixed_reviews[Ntrain:Ntrain+Nvalid]

In [200]:
len(Xtrain), len(Xvalid), len(ytrain), len(yvalid)

(160000, 40000, 160000, 40000)

In [198]:
ytrain[:10]

[4, 5, 5, 1, 5, 5, 1, 3, 1, 5]

In [172]:
yvalid[:10]

[5, 1, 2, 4, 3, 5, 5, 5, 3, 5]

In [201]:
clf = linear_model.Ridge(1.0, fit_intercept=False) # MSE + 1.0 l2

In [202]:
clf.fit(Xtrain, ytrain)

Ridge(fit_intercept=False)

In [175]:
yvalid[:10]

[5, 1, 2, 4, 3, 5, 5, 5, 3, 5]

In [203]:
theta = clf.coef_
predictions = clf.predict(Xvalid)

In [204]:
predictions[:10]

array([3.35824709, 3.9864912 , 2.69455004, 3.35824709, 1.78150423,
       3.5994451 , 2.44396229, 3.35824709, 3.15974795, 3.35824709])

In [157]:
# def MSE(predictions, labels):
#     differences = [(x-y)**2 for x,y in zip(predictions,labels)]
#     return sum(differences) / len(differences)

In [158]:
# MSE(predictions, yvalid)

2.5827738904249435

In [189]:
# predictions[:10]

array([3.32989938, 3.35035955, 3.30691597, 3.46780745, 3.2485075 ,
       3.35035955, 3.44064278, 3.35035955, 3.35035955, 3.23772555])

In [205]:
predict_labels = [1 if x > 3 else -1 for x in predictions]

In [206]:
predict_labels[:10]

[1, 1, -1, 1, -1, 1, -1, 1, 1, 1]

In [207]:
actual_labels = [d['sentiment'] for d in valid_labels]

In [208]:
actual_labels[:10]

[-1, -1, -1, 1, -1, -1, -1, 1, 1, 1]

In [209]:
correct = 0
for pred,actual in zip(predict_labels, actual_labels):
    if pred == actual:
        correct += 1 
correct / len(predict_labels)

0.60925

In [81]:
len(review_processed)

555133

In [67]:
predictions[:10]

array([4.22656409, 4.22656409, 4.22656409, 4.22656409, 4.13494545,
       4.22656409, 4.22656409, 4.22656409, 4.22656409, 4.22656409])

In [70]:
sse = 0
for ypred, actual in zip(predictions, yvalid):
    sse += (ypred-actual)**2
sse / len(yvalid)

1.3519624025606896

In [50]:
labels = [1 if prediction >=0 else -1 for prediction in predictions]

In [51]:
labels[:10]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [57]:
sum([1 for x in labels if x == -1])

1073

In [58]:
len(labels)

60000

In [55]:
correct = 0
for pred,actual in zip(labels, yvalid):
    if pred == actual:
        correct += 1
correct / len(yvalid)

0.88525

In [ ]:
wordSort = list(zip(theta[-1], words))
wordSort.sort()

In [ ]:
wordSort[:20]

In [ ]:
wordSort[-20:]

'one of the'

In [18]:
# Sentiment analysis pipeline
def pipeline(dSize = 1000, tolower=True, removePunct=True):
    wordCount = defaultdict(int)
    punctuation = set(string.punctuation)
    for d in review_data: # Strictly, should just use the *training* data to extract word counts
        r = d['text']
        if r == None: continue
        if tolower:
            r = r.lower()
            if r == None: continue
        if removePunct:
            r = ''.join(c for c in r if not c in punctuation)
            if r == None: continue
        for w in r.split():
            wordCount[w] += 1

    counts = [(wordCount[w], w) for w in wordCount]
    counts.sort()
    counts.reverse()
    
    words = [x[1] for x in counts[:dSize]]
    
    wordId = dict(zip(words, range(len(words))))
    wordSet = set(words)
    
    X = [feature(d, words, wordId, tolower, removePunct) for d in review_data]
    y = [d['rating'] for d in review_data]
    
    Ntrain,Nvalid,Ntest = int(len(review_data)*0.7),int(len(review_data)*0.2),int(len(review_data)*0.1)
    Xtrain,Xvalid,Xtest = X[:Ntrain],X[Ntrain:Ntrain+Nvalid],X[Ntrain+Nvalid:]
    ytrain,yvalid,ytest = y[:Ntrain],y[Ntrain:Ntrain+Nvalid],y[Ntrain+Nvalid:]
    
    bestModel = None
    bestVal = None
    bestLamb = None
    
    ls = [0.01, 0.1, 1, 10, 100, 1000, 10000]
    errorTrain = []
    errorValid = []

    for l in ls:
        model = sklearn.linear_model.Ridge(l)
        model.fit(Xtrain, ytrain)
        predictTrain = model.predict(Xtrain)
        MSEtrain = sum((ytrain - predictTrain)**2)/len(ytrain)
        errorTrain.append(MSEtrain)
        predictValid = model.predict(Xvalid)
        MSEvalid = sum((yvalid - predictValid)**2)/len(yvalid)
        errorValid.append(MSEvalid)
        print("l = " + str(l) + ", validation MSE = " + str(MSEvalid))
        if bestVal == None or MSEvalid < bestVal:
            bestVal = MSEvalid
            bestModel = model
            bestLamb = l
            
    predictTest = bestModel.predict(Xtest)
    MSEtest = sum((ytest - predictTest)**2)/len(ytest)
    MSEtest
    
    plt.xticks([])
    plt.xlabel(r"$\lambda$")
    plt.ylabel(r"error (MSE)")
    plt.title(r"Validation Pipeline")
    plt.xscale('log')
    plt.plot(ls, errorTrain, color='k', linestyle='--', label='training error')
    plt.plot(ls, errorValid, color='grey',zorder=4,label="validation error")
    plt.plot([bestLamb], [MSEtest], linestyle='', marker='x', color='k', label="test error")
    plt.legend(loc='best')
    plt.show()

In [19]:
pipeline(2000, False, False)

AttributeError: 'NoneType' object has no attribute 'split'

In [2]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        yield eval(l)

In [3]:
def parseData(path):
    # count = 0
    for l in gzip.open(path):
        l = json.loads(l)
        u = l['gmap_id']
        # count += 1
        yield u,l

In [4]:
allBusiness = defaultdict(dict)
for u,l in parseData("input/meta-California.json.gz"):
    allBusiness[u] = l

In [5]:
allBusiness

defaultdict(dict,
            {'0x80c2c98c0e3c16fd:0x29ec8a728764fdf9': {'name': 'City Textile',
              'address': 'City Textile, 3001 E Pico Blvd, Los Angeles, CA 90023',
              'gmap_id': '0x80c2c98c0e3c16fd:0x29ec8a728764fdf9',
              'description': None,
              'latitude': 34.0188913,
              'longitude': -118.2152898,
              'category': ['Textile exporter'],
              'avg_rating': 4.5,
              'num_of_reviews': 6,
              'price': None,
              'hours': None,
              'MISC': None,
              'state': 'Open now',
              'relative_results': ['0x80c2c624136ea88b:0xb0315367ed448771',
               '0x80c2c97cb7c52f17:0xb66ee68c1c366f2d'],
              'url': 'https://www.google.com/maps/place//data=!4m2!3m1!1s0x80c2c98c0e3c16fd:0x29ec8a728764fdf9?authuser=-1&hl=en&gl=us'},
             '0x80c2c778e3b73d33:0xbdc58662a4a97d49': {'name': 'San Soo Dang',
              'address': 'San Soo Dang, 761 S Vermont 

In [28]:
def parseData2(path):
    count = 0
    for l in gzip.open(path):
        if count % 70 == 69: # only get a subset of the data
        # if count < 10:
            d = json.loads(l)
            # u = d['user_id']
            # t = d['time']
            g = d['gmap_id']
            # r = d['rating']
            # w = d['text']
            b = allBusiness[g]
            # print(count)
            yield d
            # yield d,u,t,g,r,w,b
        count += 1

In [37]:
userIDs, itemIDs = {}, {}
interactions = []

for d in parseData2("input/review-California.json.gz"):
    u = d['user_id']
    i = d['gmap_id']
    r = d['rating']
    if r == None: continue
    if not u in userIDs: userIDs[u] = len(userIDs)
    if not i in itemIDs: itemIDs[i] = len(itemIDs)
    interactions.append((u,i,r))

In [38]:
random.shuffle(interactions)
len(interactions)

1001227

Total interactions considering None rating: 1007571

Does not consider: 1001227 (~5K less)

### Split into train and test sets

In [39]:
nTrain = int(len(interactions)*0.9)
nTest = len(interactions) - nTrain
interactionsTrain = interactions[:nTrain]
interactionsTest = interactions[nTrain:]

In [40]:
itemsPerUser = defaultdict(list)
usersPerItem = defaultdict(list)
for u,i,r in interactionsTrain:
    itemsPerUser[u].append(i)
    usersPerItem[i].append(u)

In [41]:
# Mean rating, just for initialization
mu = sum([r for _,_,r in interactionsTrain]) / len(interactionsTrain)

In [42]:
mu

4.291162840249294

#### TUNE LEARNING RATE HERE

In [90]:
optimizer = tf.keras.optimizers.legacy.Adam(0.1)

## Latent Factor Model

In [91]:
class LatentFactorModel(tf.keras.Model):
    def __init__(self, mu, K, lamb):
        super(LatentFactorModel, self).__init__()
        # Initialize to average
        self.alpha = tf.Variable(mu)
        # Initialize to small random values
        self.betaU = tf.Variable(tf.random.normal([len(userIDs)],stddev=0.001))
        self.betaI = tf.Variable(tf.random.normal([len(itemIDs)],stddev=0.001))
        self.gammaU = tf.Variable(tf.random.normal([len(userIDs),K],stddev=0.001))
        self.gammaI = tf.Variable(tf.random.normal([len(itemIDs),K],stddev=0.001))
        self.lamb = lamb

    # Prediction for a single instance (useful for evaluation)
    def predict(self, u, i):
        p = self.alpha + self.betaU[u] + self.betaI[i] +\
            tf.tensordot(self.gammaU[u], self.gammaI[i], 1)
        return p

    # Regularizer
    def reg(self):
        return self.lamb * (tf.reduce_sum(self.betaU**2) +\
                            tf.reduce_sum(self.betaI**2) +\
                            tf.reduce_sum(self.gammaU**2) +\
                            tf.reduce_sum(self.gammaI**2))
    
    # Prediction for a sample of instances
    def predictSample(self, sampleU, sampleI):
        u = tf.convert_to_tensor(sampleU, dtype=tf.int32)
        i = tf.convert_to_tensor(sampleI, dtype=tf.int32)
        beta_u = tf.nn.embedding_lookup(self.betaU, u)
        beta_i = tf.nn.embedding_lookup(self.betaI, i)
        gamma_u = tf.nn.embedding_lookup(self.gammaU, u)
        gamma_i = tf.nn.embedding_lookup(self.gammaI, i)
        pred = self.alpha + beta_u + beta_i +\
               tf.reduce_sum(tf.multiply(gamma_u, gamma_i), 1)
        return pred
    
    # Loss
    def call(self, sampleU, sampleI, sampleR):
        pred = self.predictSample(sampleU, sampleI)
        r = tf.convert_to_tensor(sampleR, dtype=tf.float32)
        return tf.nn.l2_loss(pred - r) / len(sampleR)

In [89]:
modelLFM = LatentFactorModel(mu, 5, 0.0001)

## Training step

In [92]:
def trainingStep(model, interactions):
    Nsamples = 50000
    with tf.GradientTape() as tape:
        sampleU, sampleI, sampleR = [], [], []
        for _ in range(Nsamples):
            u,i,r = random.choice(interactions)
            sampleU.append(userIDs[u])
            sampleI.append(itemIDs[i])
            sampleR.append(r)

        loss = model(sampleU,sampleI,sampleR)
        loss += model.reg()
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients((grad, var) for
                              (grad, var) in zip(gradients, model.trainable_variables)
                              if grad is not None)
    return loss.numpy()

In [93]:
for i in range(100):
    obj = trainingStep(modelLFM, interactionsTrain)
    if (i % 10 == 9): print("iteration " + str(i+1) + ", objective = " + str(obj))

iteration 10, objective = 0.7583899
iteration 20, objective = 0.76573354
iteration 30, objective = 0.75242925
iteration 40, objective = 0.75800633
iteration 50, objective = 0.75480795
iteration 60, objective = 0.74945384
iteration 70, objective = 0.74724364
iteration 80, objective = 0.75781393
iteration 90, objective = 0.7565955
iteration 100, objective = 0.7475269


In [97]:
u,i,r = interactionsTest[0]

In [98]:
modelLFM.predict(userIDs[u], itemIDs[i]).numpy()

4.403675

In [99]:
interactionsTest[0]

('101182968273849826098', '0x80dc721076f243c5:0x80479019906c26c2', 4)

In [9]:
# review_data = list(parseData2("input/review-California.json.gz"))

In [19]:
# review_data = []
# for d in readJSON2("input/review-California.json.gz"):
#     review_data.append(d)

In [17]:
# random.shuffle(review_data)

In [18]:
# review_data[0]

{'user_id': '111525652067518330708',
 'name': 'Cynthia Gonzales',
 'time': 1592095560172,
 'rating': 5,
 'text': None,
 'pics': None,
 'resp': {'time': 1592515508717,
  'text': "Cynthia, we appreciate you stopping by for a bite to eat. We'll be ready to serve another five-star meal next time you're able to make it here. - Fabulous Burgers Management"},
 'gmap_id': '0x80c2b5b8856b53d3:0x2c6d7f32d616db6f'}

In [19]:
# userIDs, itemIDs = {},{}

# for d in review_data:
#     u,i = d['user_id'], d['gmap_id']
#     if not u in userIDs: userIDs[u] = len(userIDs)
#     if not i in itemIDs: itemIDs[i] = len(itemIDs)

# nUsers, nItems = len(userIDs), len(itemIDs)

In [20]:
# nUsers, nItems

(792555, 273740)

In [21]:
# X = scipy.sparse.lil_matrix((len(review_data), nUsers + nItems))

In [22]:
# for i in range(len(review_data)):
#     user = userIDs[review_data[i]['user_id']]
#     item = itemIDs[review_data[i]['gmap_id']]
#     X[i,user] = 1 # One-hot encoding of user
#     X[i,nUsers+item] = 1 # One-hot encoding of item

In [23]:
# y = np.array([d['rating'] for d in review_data])